In [1]:
import os
print(os.getcwd())
os.chdir('/Users/boyuren/Documents/multi_head_graph_rag/MH-GRAG-V1')
print(os.getcwd())

/Users/boyuren/Documents/multi_head_graph_rag/MH-GRAG-V1/tests
/Users/boyuren/Documents/multi_head_graph_rag/MH-GRAG-V1


# 测试整体RAG查询流程

In [ ]:
# 数据加载

import pandas as pd
from src.utils.embedding_utils import load_df_from_csv_with_embedding
from src.utils.config import OUTPUT_PATH

entities_with_embeddings=load_df_from_csv_with_embedding(f"{OUTPUT_PATH}/multihop_rag/final_entities_with_embeddings_by_chatgpt4o.csv",['entity_embedding'])
entities_with_clusters=pd.read_csv("export/multihop_rag/filtered_entities_cluster_reults.csv")
relationships=pd.read_csv("export/multihop_rag/filtered_relationships_with_duplicates.csv")

In [6]:
from src.utils.embedding_utils import find_k_nearest_entities


# 获取查询嵌入
query = "Who is the figure associated with generative AI technology whose departure from OpenAI was considered shocking according to Fortune, and is also the subject of a prevailing theory suggesting a lack of full truthfulness with the board as reported by TechCrunch?"

# 查询最相近的 k 个实体
k = 1
nearest_entities = find_k_nearest_entities(query, entities_with_embeddings, k=k)
nearest_entities

,entity_name,entity_type,description,entity_id,corpus_id,entity_embedding,similarity
3904,TECHCRUNCH,ORGANIZATION,TechCrunch is a technology news website that r...,3904,406,"[0.008506687358021736, -0.013194956816732883, ...",0.856984


In [7]:
from src.report.community_report import generate_and_merge_reports

entity_id=nearest_entities["entity_id"].iloc[0]

merge_reports=generate_and_merge_reports(entity_id, entities_with_clusters, relationships, output_file=f'export/rag_test/merged_community_report_{entity_id}.json')

No related relationships found for entity_id 3904 in cluster 'multihead_leiden_cluster_head_1'. Skipping...
No related relationships found for entity_id 3904 in cluster 'multihead_leiden_cluster_head_2'. Skipping...
No related relationships found for entity_id 3904 in cluster 'multihead_leiden_cluster_head_3'. Skipping...
Merged report saved to export/rag_test/merged_community_report_3904.json


In [10]:
from src.utils.message_templates import generate_query_template
from src.utils.llm_helpers import call_llm_api
import json

# # 加载生成的社区报告
# def load_report(file_path):
#     """
#     加载 JSON 格式的社区报告。
#     """
#     with open(file_path, 'r') as file:
#         return json.load(file)
# merge_reports= load_report('export/rag_test/merged_community_report.json')

query_template=generate_query_template(query,merge_reports)
answer=call_llm_api(query_template)

In [11]:
import json

def parse_llm_answer(answer):
    """
    解析 LLM 返回的 JSON 格式答案，并提取其中的内容。
    
    参数：
        answer (str): LLM 返回的 JSON 字符串。

    返回：
        dict: 包含解析后的答案和所用发现编号。
    """
    try:
        # 清理字符串，去除外部的 ```json ``` 包裹
        cleaned_answer = answer.strip("```").replace("json\n", "")
        
        # 将字符串转换为 Python 字典
        parsed_answer = json.loads(cleaned_answer)
        
        # 提取答案和所用发现
        answer_text = parsed_answer.get("answer", "No answer found.")
        used_findings = parsed_answer.get("used_findings", [])

        return {"answer": answer_text, "used_findings": used_findings}
    
    except json.JSONDecodeError as e:
        print(f"JSONDecodeError: {e}")
        return None
    
parsed_result = parse_llm_answer(answer)

if parsed_result:
    print("Answer:", parsed_result["answer"])
    print("Used Findings:", parsed_result["used_findings"])

Answer: The figure associated with generative AI technology whose departure from OpenAI was considered shocking according to Fortune is Sam Altman. He is also the subject of a prevailing theory suggesting a lack of full truthfulness with the board as reported by TechCrunch.
Used Findings: [1, 3]
